- Available approaches:
    - Closed form solution:
        - Requires data to be Gaussian distributed with equal covariance
    - Linear regression approach:
        - Recall what we did in Linear Regression:
            - Take derivative of cost function (OLS), set to 0, solve for weights
            - Can't do this with logistic regression's cross-entropy loss
            - You can get the derivative, but not solve for weights (try this)
    - Gradient Descent
    
### Gradient Descent

<table>
<tr>
    <td><img src='GradientDescent.png'></td>
    <td><img src='GradientDescent2.png'></td>
</tr>
</table>
- Idea: Take small steps in the direction of derviative
- Step size is learning rate
- e.g. w = -2
- w = -2 - 1 * (-1) - 1
- Now we're closer to the optimal point (w = 0)
- The slope is 0 at the bottom, so no more changes will occur
- This works for any objective function, as long as you can calculate the derivative.
- How do you choose the learning rate? No scientific method yet. Try different values.

### Gradient Descent for logistic regression

$J = - \sum_{n=1}^{N} t_n * log(y_n) + (1 - t_n) * log (1 - y_n)$

Split into 3 derivatives:  
$\frac{\partial J}{\partial w_i} = 
           - \sum_{n=1}^{N} \frac{\partial J}{\partial y_n}
                            \frac{\partial y_n}{\partial a_n}
                            \frac{\partial a_n}{\partial w_i}$

$\frac{\partial J}{\partial y_n} = t_n \frac{1}{y_n} + (1 - t_n) \frac{1}{1 - y_n} (-1) = \frac{t_n}{y_n} - \frac{(1 - t_n)}{1 - y_n}$

$y_n = \sigma({a_n}) = \frac{1}{1 + exp(-a_n)}$

$ \frac{\partial y_n}{\partial a_n} = 
            \frac{-1}{1 + exp(-a_n)} exp(-a_n) (-1) 
           = \frac{exp(-a_n)}{(1 + exp(-a_n)^2}
           = \frac{1}{1 + exp(-a_n)} \frac{exp(-a_n)}{1 + exp(-a_n)}
           = y_n (1 - y_n) $
           
$a_n = w^T x_n = w_0 x_{n0} + w_1 x_{n1} + w_2 x_{n2} + ...$

$\frac{\partial a_n}{\partial w_i} = x_{ni}$

Put them together:  
$ \frac{\partial J}{\partial w_i} =  - \sum_{n=1}^{N} \frac{t_n}{y_n} y_n (1 - y_n) x_{ni} -  \frac{(1 - t_n)}{1 - y_n} y_n (1 - y_n) x_{ni} $

$\frac{\partial J}{\partial w_i} = \sum_{n=1}^{N} (y_n - t_n) x_{ni}$

Vectorize:  
$\frac{\partial J}{\partial w} = \sum_{n=1}^{N} (y_n - t_n) x_n = X^T(Y - T)$

Bias Term:  
$\frac{\partial J}{\partial w_0} = \sum_{n=1}{N} (y_n - t_n) x_{n0} = \sum_{n=1}{N} (y_n - t_n) $  
(because the $x_0$s are 1 (weights change, $x$ doesn't))

In [18]:
import numpy as np

N = 100
D = 2

# Gaussian centered at 0 (std = 1)
X = np.random.randn(N, D)

# Clever trick: Centering at (-2, -2)
X[:50, :] = X[:50, :] - 2 * np.ones((50, D))

# Centering at (2, 2)
X[50:, :] = X[50:, :] + 2 * np.ones((50, D))

# Array of targets
# First 50 at 0 and next 50 at 1
T = np.array([0]*50 + [1]*50).T 

ones = np.array([[1] * N]).T
Xb = np.concatenate((ones, X), axis=1)
w = np.random.randn(D + 1)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Calculate the model output
z = Xb.dot(w)
Y = sigmoid(z)

def cross_entropy(T, Y):
    return - (T * np.log(Y) + (1 - T) * np.log(1 - Y)).sum()

learning_rate = 0.1
for i in range(100):
    if i % 10 == 0:
        print(cross_entropy(T, Y))
        
    # Here is where you use what we derived above
    w += learning_rate * np.dot((T - Y).T, Xb)
    Y = sigmoid(Xb.dot(w))
    
print("Final w:", w)

25.9741949381
1.30758366212
0.722971041569
0.537385169235
0.455394576041
0.408661150934
0.376405876741
0.351251524457
0.330261143947
0.312103241135
Final w: [-3.07998631  2.86749167  5.43367456]


In [20]:
# Compare with closed form solution:
w2 = np.array([0, 4, 4])
# We're far away yet
# More iterations?

w = np.random.randn(D + 1)
z = Xb.dot(w)
Y = sigmoid(z)

learning_rate = 0.1
for i in range(150):
    if i % 10 == 0:
        print(cross_entropy(T, Y))
        
    # Here is where you use what we derived above
    w += learning_rate * np.dot((T - Y).T, Xb)
    Y = sigmoid(Xb.dot(w))
    
print("Final w:", w)

11.198603289
1.59693764508
1.15167791468
0.922210128028
0.776350382614
0.673743481155
0.59695800392
0.537014206229
0.488746327727
0.448945277639
0.415500488732
0.386961130273
0.36229396171
0.340741467845
0.321734632297
Final w: [-3.0627943   2.82476522  5.37199592]


In [21]:
# Almost seems like a local minimum here eh,
#    although it doesn't make sense because I'm pretty sure it's a convex
#    objective.
# Compare with closed form solution:
w2 = np.array([0, 4, 4])
# We're far away yet
# More iterations?

w = np.random.randn(D + 1)
z = Xb.dot(w)
Y = sigmoid(z)

learning_rate = 0.1
for i in range(150):
    if i % 10 == 0:
        print(cross_entropy(T, Y))
        
    w += learning_rate * np.dot((T - Y).T, Xb)
    Y = sigmoid(Xb.dot(w))
    
print("Final w:", w)

304.534078262
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Final w: [ -6.16616321  11.64989518  20.05007614]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply


In [26]:
# And what's with these nan errors?

def cross_entropy2(T, Y):
    E = 0
    for i in range(N):
        if T[i] == 1:
            E -= np.log(Y[i])
        else:
            E -= np.log(1 - Y[i])
    return E

w = np.random.randn(D + 1)
z = Xb.dot(w)
Y = sigmoid(z)

learning_rate = 0.1
for i in range(200):
    if i % 10 == 0:
        print(cross_entropy2(T, Y))
        
    w += learning_rate * np.dot((T - Y).T, Xb)
    Y = sigmoid(Xb.dot(w))
    
print("Final w:", w)

62.6052910996
1.04732203569
0.514008307598
0.339789567136
0.256415093052
0.207187514235
0.174585801322
0.151417300646
0.134149346571
0.120828213424
0.110279357124
0.101751646517
0.0947412661633
0.0888972058543
0.0839672191938
0.0797653048518
0.0761512980752
0.0730176012487
0.0702802911945
0.0678730011559
Final w: [-4.81795914  4.88905143  8.81262154]


In [58]:
# Almost seems like a local minimum here eh,
#    although it doesn't make sense because I'm pretty sure it's a convex
#    objective.
# Compare with closed form solution:
w2 = np.array([0, 4, 4])
# We're far away yet
# More iterations?

w = np.random.randn(D + 1)
z = Xb.dot(w)
Y = sigmoid(z)

learning_rate = 0.1
for i in range(150):
    if i % 10 == 0:
        print(cross_entropy(T, Y))
        
    w += learning_rate * np.dot((T - Y).T, Xb)
    Y = sigmoid(Xb.dot(w))
    
print("Final w:", w)

77.9337472311
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Final w: [-4.48606932  4.79089743  8.61609692]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply



The nan problem:
Clearly the problem is $np.log(0)$ somewhere in the problem based on the warning. 

In the vectorized cross_entropy function, I'm calculating $np.log(1 - Y)$ and $np.log(Y)$ on the entire vector $Y$, including potential predictions of 0 and 1, which will result in $nan$ with $np.log$. The non-vectorized implementation calculates $np.log(y_i)$ or  $np.log(y_i)$ only if $t_i == 1$ or $t_i == 0$ respectively. In both cases the operand is closer to 1 than 0. This problem can occur even with the non-vector function if for some reason the classifier is bad enough to predict 0 for a target = 1 or vice versa.

In [65]:
# And what's with these nan errors?

def cross_entropy2(T, Y):
    E = 0
    for i in range(N):
        if T[i] == 1:
            E -= np.log(Y[i])
        else:
            E -= np.log(1 - Y[i])
    return E

w = np.random.randn(D + 1)
z = Xb.dot(w)
Y = sigmoid(z)

learning_rate = 0.01
epochs = 200
for i in range(epochs):
    if i % 10 == 0:
        print(cross_entropy2(T, Y))
        
    w += learning_rate * np.dot((T - Y).T, Xb)
    Y = sigmoid(Xb.dot(w))
    
print("Final w:", w, "epochs =", epochs)

72.2226887599
5.01351232036
3.88079676329
3.2862313922
2.89681455625
2.61430078632
2.39668775813
2.22223223881
2.07827128374
1.95681852289
1.85253958716
1.76171367578
1.68165565833
1.61037359406
1.54635542276
1.48843066962
1.43567777587
1.38736026271
1.34288170995
1.30175335088
Final w: [-1.13437684  1.66408505  3.03458363] epochs = 200
